In [ ]:
import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans

if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    EE_PROJECT_ID = userdata.get('EE_PROJECT_ID') 
else:
    from dotenv import load_dotenv
    import os
    load_dotenv()  # take environment variables
    EE_PROJECT_ID = os.getenv('EE_PROJECT_ID')

# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID) #<- Remember to change this to your own project's name!

# A Sentinel-2 surface reflectance image.
img = ee.Image('COPERNICUS/S2_SR/20210109T185751_20210109T185931_T10SEG')
true_color_viz = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 2700,
    'gamma': 1.3,
}

m = geemap.Map()
m.set_center(-122.36, 37.47, 10)
m.add_layer(img, true_color_viz, 'Sentinel-2 image')

# # Create a Boolean land mask from the SWIR1 band water is value 0, land is 1.
land_mask = img.select('B11').gt(100)
# m.add_layer(land_mask, {'palette': ['blue', 'lightgreen']}, 'Land mask')

# Mask the land mask by itself pixel values equal to 0 (water) become invalid.
land_mask_masked = land_mask.selfMask()

land_mask_interior = img.select('B11').gt(100)
land_mask_interior_marked = land_mask_interior.focal_min(20).selfMask()
# land_mask_clear_marked = land_mask_interior.focal_max(20).Not().selfMask()
land_mask_clear_marked = land_mask_interior.focal_max(20).And(land_mask_interior.Not())


m.add_layer(land_mask_interior_marked, {'palette': ['red']}, 'Land mask, interior')
m.add_layer(land_mask_clear_marked, {'palette': ['green']}, 'Land mask, clear')
m.add_layer(land_mask_masked, {'palette': ['gold']}, 'Land mask, masked')

m